In [3]:
from ollama import Client

I am using the open source `ollama` to run the model locally.

In [6]:
client = Client()

In [15]:
output = client.generate(
    model='llama3.2:3b',
    prompt="The capital of France is",
    options={
        'num_tokens': 100
    }
)

print(output.response)

Paris.


Chat method

In [13]:
output = client.chat(
    model='llama3.2:3b',
    messages=[{
        'role': 'user',
        'content': 'The capital of France is'
    }],
    options={
        'num_tokens': 100
    }
)


TypeError: 'ProxyClientChat' object is not callable